# 1. Load Dataset

## Audio data
    - wav data와 이와 매칭되는 json label data를 model input이 가능하게 변환 필요
    - WhisperFeatureExtractor는 16kHz로 sampling된 데이터를 입력받는다


In [57]:
# set GPU
# torch 보다 먼저 import 해야 아래 사항이 적용된다
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "1"  # Set the GPU 1 to use
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [60]:
import glob
from datasets import load_dataset

# File path
training_file_dir = "/data/freetalk_senior/1.Training/"

training_raw_data = sorted(glob.glob(os.path.join(training_file_dir, "raw_data/**/*.zip"), recursive=True))
training_raw_data = training_raw_data[8] # 앞의 2개 뺌

training_labeled_data = []
training_labeled_data.append(sorted(glob.glob(os.path.join(training_file_dir, "labeled_data/3.*/**/*.json"), recursive=True)))


In [61]:
training_raw_data

'/data/freetalk_senior/1.Training/raw_data/3.스튜디오/3.스튜디오_자유대화(노인남여)_TRAINING.zip'

In [62]:
len(training_labeled_data[0])

133387

In [63]:
data_files = {
    "train_input": training_raw_data,    
    "train_label": training_labeled_data,
}

In [64]:

# Load Dataset
from datasets import Features, Audio

# load_dataset 이 새로 만들어 버리는 label column 때문에 에러 발생하여 features명시
features = Features({
    'audio': Audio()
})

train_input = load_dataset("audiofolder", data_files={"train": data_files["train_input"]}, 
                                            features=features)
train_input

DatasetDict({
    train: Dataset({
        features: ['audio'],
        num_rows: 133387
    })
})

In [66]:
import datasets 

train_label = load_dataset("json", data_files=data_files["train_label"][0])

train_dataset = datasets.DatasetDict({
    "train": datasets.concatenate_datasets([train_input['train'], train_label['train']], axis=1)
})

train_dataset

Resolving data files:   0%|          | 0/133387 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio', '발화정보', '대화정보', '녹음자정보'],
        num_rows: 133387
    })
})

In [67]:
senior_dataset = train_dataset.shuffle(seed=44)

In [80]:
senior_dataset.column_names["train"][2:4]

['대화정보', '녹음자정보']

In [82]:
senior_dataset = senior_dataset.map(remove_columns=senior_dataset.column_names["train"][2:4])

In [85]:
senior_dataset = senior_dataset.map()

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (1647170855.py, line 1)

# 2. Set Toknizer & FeatureExtractor

## Feature Extractor: Audio raw data에 대 preprocessing 한다
    - Transformers.WhisperFeatureExtractor
    1. Audio sample에 대해 padding 및 truncating을 해 모든 sample길이를 30초 맞춘다.
        whisper model은 attention mask없이 발화 신호로부터 무시해야 할 input을 직접 추론하도록 학습되었다
    2. Audio array를 log-Mel spectrum으로 변환. -> 이 것이 model input

## Model: seq-to-seq mapping

## Tokeninzer: model output을 text format으로 postprocessing
    - Transformers.WhisperTokenizer


In [72]:
checkpoint = "jiwon65/whisper-small_korean-zeroth"

# Whisper model output은 vocabulary 단어의 index
# 이를 실제 문자와 mapping하기 위해 Tokenizer 사용

from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(checkpoint, language="Korean", task="transcribe")
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(checkpoint, language="Korean", task="transcribe")

# Feature Extractor
from transformers import WhisperFeatureExtractor

# jiwon65/whisper-small_korean-zeroth model을 fine tuning
feature_extractor = WhisperFeatureExtractor.from_pretrained(checkpoint)
# feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [69]:
# WhisperProcessor: 위 두 모듈을 하나로 묶는다
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(checkpoint, language="Korean", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# 3. Dataset Preprocessing

In [84]:
def preprocess_dataset(batch):
    audio=batch["audio"]
    label=batch["발화정보"]
    
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    batch["labels"] = tokenizer(label['stt']).input_ids
    return batch

senior_dataset = senior_dataset.map(preprocess_dataset, 
                                    remove_columns=senior_dataset.column_names["train"], 
                                    # batched=True,
                                    # batch_size=1,
                                    # num_proc=os.cpu_count()
                                   )

Map:   0%|          | 0/133387 [00:00<?, ? examples/s]

KeyboardInterrupt: decoding with 'cp437' codec failed (KeyboardInterrupt: )

In [7]:
splited_senior_dataset = senior_dataset["train"].train_test_split(test_size=0.2)

In [8]:
splited_senior_dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 214
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 54
    })
})

# 4. Set Trainer & Training

## 4-1. Set DataCollator

    seq-to-seq model을 위한 DataCollator는 input_feature와 label을 독립적으로 다룬다는 점에서 특징적.
        input_feature -> FeatureExtractor
        label -> Tokenizer 로 다뤄야 함

In [9]:
# input_feature -> log_mel_spectrogram -> PyTorch Tensor
import torch
from dataclasses import dataclass
from typing import Any, Dict, List, Union

# DatasetDict -> torch.tensor
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # 인풋 데이터와 라벨 데이터의 길이가 다르며, 따라서 서로 다른 패딩 방법이 적용되어야 한다. 그러므로 두 데이터를 분리해야 한다.
        # 먼저 오디오 인풋 데이터를 간단히 토치 텐서로 반환하는 작업을 수행한다.
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")
        
        # Tokenize된 레이블 시퀀스를 가져온다.
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # 레이블 시퀀스에 대해 최대 길이만큼 패딩 작업을 실시한다.
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        
        # 패딩 토큰을 -100으로 치환하여 loss 계산 과정에서 무시되도록 한다.
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)
        
        # 이전 토크나이즈 과정에서 bos 토큰이 추가되었다면 bos 토큰을 잘라낸다.
        # 해당 토큰은 이후 언제든 추가할 수 있다.
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        
        batch["labels"] = labels
        
        return batch

In [10]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(checkpoint, language="Korean", task="transcribe")

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 4-2. Set metric

In [11]:
import evaluate

metric = evaluate.load('cer')

In [12]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    label_ids[label_ids == -11] = tokenizer.pad_token_id

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    cer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

## 4-3. Set Train args

    output_dir: model weight를 저장하는 경로
    generation_max_length: evaluation동안 recursive하게 생성되는 token들의 최대 길이 
    save_steps: Training동안, 이 parameter에 설정한 step 마다 checkpoint 저장
    eval_steps: Training동안, 이 parameter에 설정한 step 마다 checkpoint 평가 수행
    report_to: training_log를 어디 저장할 지 설정. default: tensorboard

In [13]:
# pretrained_model load
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(checkpoint)

/home/wooseok/anaconda3/envs/whisper_kor/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [14]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [15]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./test_trainer",  # 원하는 리포지토리 이름을 임력한다.
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,  
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=1000,  # epoch 대신 설정
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=400,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="cer",  # 한국어의 경우 'wer'보다는 'cer'이 더 적합할 것
    greater_is_better=False,
    push_to_hub=False,
)

## 4-4. Set Trainer

In [16]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=splited_senior_dataset["train"],
    eval_dataset=splited_senior_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

/home/wooseok/anaconda3/envs/whisper_kor/lib/python3.9/site-packages/accelerate/accelerator.py:444: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


## 4-5. Set GPU

In [17]:
# 맨 처음 set

In [18]:
print("CUDA_DEVICE_ORDER:", os.environ.get("CUDA_DEVICE_ORDER"))
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

CUDA_DEVICE_ORDER: PCI_BUS_ID
CUDA_VISIBLE_DEVICES: 1


In [19]:
# torch 를 먼저 import 하면 CUDA_VISIBLE_DEVICES 설정이 안먹음
import torch
print("Number of GPUs available:", torch.cuda.device_count())

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

model.config.use_cache = False # mem usage 줄이기 위함

Number of GPUs available: 1


In [20]:
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda:0
Current cuda device: 0
Count of using GPUs: 1


In [21]:
torch.cuda.empty_cache()

trainer.train() # data 300개로 했더니 너무 overfitting 됐다!

/home/wooseok/anaconda3/envs/whisper_kor/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Cer
1000,0.000300,0.732618,23.448276


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [], 'begin_suppress_tokens': [220, 50257]}
There were missing keys in the checkpoint model loaded: ['proj_out.weight'].


TrainOutput(global_step=1000, training_loss=0.05231386368442327, metrics={'train_runtime': 1468.5282, 'train_samples_per_second': 5.448, 'train_steps_per_second': 0.681, 'total_flos': 2.30868320256e+18, 'train_loss': 0.05231386368442327, 'epoch': 37.38})

In [22]:
print(torch.__version__)
print(torch.version.cuda)

2.3.1+cu121
12.1


In [24]:
trainer.args.eval_steps = 400

In [26]:
trainer.evaluate()

{'eval_loss': 0.7326184511184692,
 'eval_cer': 23.448275862068964,
 'eval_runtime': 18.2983,
 'eval_samples_per_second': 2.951,
 'eval_steps_per_second': 0.383,
 'epoch': 37.38}